# MNIST Neural Network
* MNIST 데이터셋을 이용해 Neural Network를 학습시키고 이미지 분류를 풀어보도록 하겠습니다.

In [7]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.1.0


## Train Data & Test Data

In [8]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

# shape 확인
print(x_train.shape)
print(y_train.shape)

# 데이터 channel 추가
x_train = np.expand_dims(x_train, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
x_test = np.expand_dims(x_test, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

# 데이터 pixel 값을 0 ~ 1 사이로 조정 (Normalization)
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

# 레이블 -> One-Hot encoding 변환
y_train = to_categorical(y_train, 10) # [N,] -> [N, 10]
y_test = to_categorical(y_test, 10) # [N,] -> [N, 10]

# shape 확인
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)
(60000, 28, 28, 1)
(60000, 10)


##  Tensorflow Keras
### MNIST 데이터셋을 이용하여 이미지를 분류하기 위한 모델을 만들도록 하겠습니다
### Keras를 이용하여 Neural Network 모델을 만든다.

### Keras Sequential API

In [9]:
model = tf.keras.Sequential()

# Input Layer
model.add(tf.keras.Input(shape=(28, 28, 1)))
model.add(tf.keras.layers.Flatten())

# Hidden Layer 1
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

# Hidden Layer 2
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

# Output Layer
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))


model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
batch_normalization_8 (Batch (None, 512)               2048      
_________________________________________________________________
activation_12 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               262656    
_________________________________________________________________
batch_normalization_9 (Batch (None, 512)               2048      
_________________________________________________________________
activation_13 (Activation)   (None, 512)              

### Keras Functional API

In [10]:
# Input Layer
inputs = tf.keras.Input(shape=(28, 28, 1))
flatten = tf.keras.layers.Flatten()(inputs)

# Hidden Layer 1
dense1 = tf.keras.layers.Dense(512)(flatten)
bn1 = tf.keras.layers.BatchNormalization()(dense1)
relu1 = tf.keras.layers.Activation('relu')(bn1)

# Hidden Layer 2
dense2 = tf.keras.layers.Dense(512)(relu1)
bn2 = tf.keras.layers.BatchNormalization()(dense2)
relu2 = tf.keras.layers.Activation('relu')(bn2)

# Output Layer
dense3 = tf.keras.layers.Dense(10)(relu2)
logits = tf.keras.layers.Activation('softmax')(dense3)

model = tf.keras.Model(inputs=inputs, outputs=logits)

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               401920    
_________________________________________________________________
batch_normalization_10 (Batc (None, 512)               2048      
_________________________________________________________________
activation_15 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               262656    
_________________________________________________________________
batch_normalization_11 (Batc (None, 512)               2048

### Keras Subclassing

In [11]:
class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(512)
    self.bn1 = tf.keras.layers.BatchNormalization()
    self.relu1 = tf.keras.layers.Activation('relu')
    self.dense2 = tf.keras.layers.Dense(512)
    self.bn2 = tf.keras.layers.BatchNormalization()
    self.relu2 = tf.keras.layers.Activation('relu')
    self.dense3 = tf.keras.layers.Dense(10)
    self.logits = tf.keras.layers.Activation('softmax')

  def call(self, inputs, training=False):
    net = self.flatten(inputs)
    net = self.dense1(net)
    net = self.bn1(net)
    net = self.relu1(net)
    net = self.dense2(net)
    net = self.bn2(net)
    net = self.relu2(net)
    net = self.dense3(net)
    net = self.logits(net)
    return net

model = MyModel()
temp_inputs = tf.keras.Input(shape=(28, 28, 1))
model(temp_inputs)
model.summary()

Model: "my_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
batch_normalization_12 (Batc (None, 512)               2048      
_________________________________________________________________
activation_18 (Activation)   (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 512)               262656    
_________________________________________________________________
batch_normalization_13 (Batc (None, 512)               2048      
_________________________________________________________________
activation_19 (Activation)   (None, 512)               0

## Model을 훈련시키기 위해 손실 함수와 최적화 함수 설정

In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 훈련

In [13]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 17s 287us/sample - loss: 0.1862 - accuracy: 0.9434
Epoch 2/5
60000/60000 [==============================] - 16s 267us/sample - loss: 0.0961 - accuracy: 0.9700
Epoch 3/5
60000/60000 [==============================] - 17s 280us/sample - loss: 0.0695 - accuracy: 0.9776
Epoch 4/5
60000/60000 [==============================] - 17s 291us/sample - loss: 0.0563 - accuracy: 0.9815
Epoch 5/5
60000/60000 [==============================] - 17s 277us/sample - loss: 0.0466 - accuracy: 0.9843


## Predict (예측)

In [14]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 129us/sample - loss: 0.0598 - accuracy: 0.9813


[0.059753484327782644, 0.9813]